<a href="https://colab.research.google.com/github/Steven-Macas/deepLearning_SAA/blob/main/deep_learning_parte1_steven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf

In [9]:
!unzip -n '/content/drive/MyDrive/Colab Notebooks/DL_flowers.zip' >> /dev/null

In [11]:
data_path = 'flowers/'
imgs_path = data_path+"images/"
labels_path = data_path + "labels.csv"

In [12]:
labels = pd.read_csv(labels_path, dtype = {"class": "category"})

In [13]:
seed = 0
train_data, test_data = train_test_split(labels, test_size=0.3, random_state = seed)
dev_data, test_data = train_test_split(test_data, test_size = 0.5, random_state=seed)

In [14]:
train_data = train_data.reset_index(drop=True)
dev_data = dev_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [15]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [16]:
classes = ["0","1","2","3","4"]
n_classes = len(classes)
img_width = img_height = 224
x_col = 'image_name'
y_col = 'class'

In [17]:
batch_size = 128
train_generator = datagen.flow_from_dataframe (
  dataframe=train_data, directory=imgs_path, x_col=x_col, y_col=y_col,
  has_ext=True, class_mode="categorical", target_size=(img_width,img_height),
  batch_size=batch_size, classes=classes)

dev_generator = datagen.flow_from_dataframe (
  dataframe=dev_data, directory=imgs_path, x_col=x_col, y_col=y_col,
  has_ext=True, class_mode="categorical", target_size=(img_width,img_height),
  batch_size=batch_size, classes=classes)

test_generator = datagen.flow_from_dataframe (
  dataframe=test_data, directory=imgs_path, x_col=x_col, y_col=y_col,
  has_ext=True, class_mode="categorical", target_size=(img_width,img_height),
  batch_size=batch_size, classes=classes)

Found 3020 validated image filenames belonging to 5 classes.
Found 647 validated image filenames belonging to 5 classes.
Found 648 validated image filenames belonging to 5 classes.


In [18]:
base_model = tf.keras.applications.InceptionV3(
    weights = 'imagenet', include_top = False, input_shape = (img_width, img_height,3)
)

87910968/87910968 [==============================] - 3s 0us/step


In [19]:
for layer in base_model.layers:
  layer.trainable = False

In [26]:
# Añadimos nuevas capas al final para adaptar el modelo a nuestro problemas
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Añadimos una última capa completamente conectada con 5 neuronas
# (número de clases) para obtener la salida de la red
predictions = tf.keras.layers.Dense(n_classes, activation='softmax')(x)

In [27]:
# Creamos el modelo final y lo compilamos
model = tf.keras.models.Model(inputs=[base_model.input], outputs=[predictions])
model.summary() # representación en modo texto del modelo

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [28]:
model.compile(
loss='categorical_crossentropy', # función de pérdida para problemas de clasificación multiclase
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # optimizador Adam, learning rate
metrics=['accuracy'])

In [29]:
# Entrenamos el modelo con los datos preparados
model.fit(
  train_generator,
  epochs=10, # numero de epochs
  verbose=2, # muestra informacion del error al finalizar cada epoch
  steps_per_epoch=len(train_data)/batch_size,
  validation_data=dev_generator,
  validation_steps=len(dev_data)/batch_size)

Epoch 1/10
23/23 - 345s - loss: 1.5148 - accuracy: 0.3609 - val_loss: 1.2849 - val_accuracy: 0.5070 - 345s/epoch - 15s/step
Epoch 2/10
23/23 - 360s - loss: 1.1580 - accuracy: 0.5778 - val_loss: 1.0424 - val_accuracy: 0.6229 - 360s/epoch - 15s/step
Epoch 3/10
23/23 - 332s - loss: 0.9505 - accuracy: 0.6881 - val_loss: 0.8985 - val_accuracy: 0.6878 - 332s/epoch - 14s/step
Epoch 4/10
23/23 - 360s - loss: 0.8207 - accuracy: 0.7331 - val_loss: 0.8067 - val_accuracy: 0.7202 - 360s/epoch - 15s/step
Epoch 5/10
23/23 - 330s - loss: 0.7348 - accuracy: 0.7599 - val_loss: 0.7422 - val_accuracy: 0.7496 - 330s/epoch - 14s/step
Epoch 6/10
23/23 - 355s - loss: 0.6710 - accuracy: 0.7851 - val_loss: 0.6924 - val_accuracy: 0.7651 - 355s/epoch - 15s/step
Epoch 7/10
23/23 - 354s - loss: 0.6225 - accuracy: 0.8020 - val_loss: 0.6546 - val_accuracy: 0.7774 - 354s/epoch - 15s/step
Epoch 8/10
23/23 - 354s - loss: 0.5847 - accuracy: 0.8132 - val_loss: 0.6222 - val_accuracy: 0.7867 - 354s/epoch - 15s/step
Epoch 9/

In [30]:
# Por ultimo, podemos evaluar el modelo en el conjunto de test
test_loss, test_acc = model. evaluate(
test_generator,
steps=len(test_data)/batch_size,
verbose=1)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

5/5 [==============================] - 59s 12s/step - loss: 0.5553 - accuracy: 0.8225 
test_loss: 0.5553, test_acc: 0.8225


# Conclusión

---



Puntos fuertes:

Incremento progresivo de la precisión: Durante las 10 épocas de entrenamiento, se observa un incremento progresivo en la precisión tanto en el conjunto de entrenamiento como en el de validación. Esto indica que el modelo está aprendiendo y mejorando su capacidad de hacer predicciones precisas.
Precisión aceptable: El modelo alcanza una precisión del 82.25% en el conjunto de prueba, lo cual es un resultado prometedor y muestra que el modelo es capaz de realizar clasificaciones correctas en una proporción significativa de casos.

Puntos débiles:

Tiempo de entrenamiento prolongado: El tiempo requerido para entrenar la red neuronal es relativamente largo, lo cual podría ser problemático en escenarios donde se necesita un entrenamiento rápido o se tienen limitaciones de tiempo.

En conclusión, la red neuronal desarrollada muestra buenos resultados en términos de precisión en el conjunto de prueba. Sin embargo, es importante considerar el tiempo requerido para entrenar el modelo y la necesidad de ajustar adecuadamente los hiperparámetros. 